In [1]:
import yaml
import src.blocks as blocks
from src import Block
import src.merge as merge
import copy
from collections import OrderedDict
from jinja2 import Environment, FileSystemLoader

In [2]:
# Read configuration
with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config


{'gen1': {'a': 2, 'b': 10, 'c': 4}, 'gen2': {'d': 0.5}}

In [3]:
# Read master
with open("master.yaml", "r") as f:
    master = yaml.load(f, Loader=yaml.FullLoader)


In [4]:
for gen in sorted(master.keys()):
    print("Now browsing generation: " + gen)
    for block in master[gen]['script']:
        print("Now browsing block: " + block)
        print(master[gen]['script'][block])
    print()

Now browsing generation: gen1
Now browsing block: multiply
{'args': ['b', 'c'], 'output': 'bc'}
Now browsing block: add
{'args': ['a', 'bc'], 'output': 'a_bc'}
Now browsing block: factorial
{'args': 'a_bc', 'output': 'fact_a_bc'}



In [5]:
def generate_main(script, config, dict_blocks, save = None, dict_imports_merge = None):

    # Generate header
    header_str = "def main():\n"

    # Declare parameters
    parameters_str = "\t# Declare parameters\n"
    for param, value in config.items():
        parameters_str += "\t" + param + " = " + str(value) + "\n"

    # Declare blocks
    blocks_str = "\n\t# Declare blocks\n"
    for block, dict_block in script.items():
        # Get block arguments
        l_args = dict_block["args"]
        if not isinstance(l_args, list):
            l_args = [l_args]
        # Get block outputs
        l_outputs = dict_block["output"]
        if not isinstance(l_outputs, list):
            l_outputs = [l_outputs]
        # Write blocks string
        blocks_str += "\t" + dict_blocks[block].get_assignation_call_str(l_args, l_outputs) + "\n"

    # Save output
    if save is not None:
        if dict_imports_merge is not None:
            save_str = "\n\t# Save output\n"
            for output, save_method in save.items():
                if save_method == 'npy':
                    save_str += f"\t{dict_imports_merge['numpy']}.save('{output}', {output})\n"
                elif save_method == 'pickle':
                    save_str += f"\twith open('{output}', 'wb') as f:\n"
                    save_str += f"\t\t{dict_imports_merge['pickle']}.dump({output}, f)\n"
        else:
            raise ValueError("dict_imports_merge must be provided to save output")           

    main_str = header_str + parameters_str + blocks_str + save_str

    # Replace tabs by spaces to prevent inconsistent indentation
    main_str = main_str.replace("\t", "    ")

    return main_str

In [6]:
# Generate generations from template
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("default_template.txt")

for gen in sorted(master.keys()):
    file_path = f"{gen}.py"
    print("Now browsing generation: " + gen)
    print("\n")
    dict_blocks = OrderedDict()
    for block in master[gen]["script"]:
        dict_blocks[block] = getattr(blocks, block)

    # Merge all imports
    dict_imports_merge = merge.merge_imports(list(dict_blocks.values()))

    # Add the imports for save methods
    if "save" in master[gen]:
        for save_method in master[gen]["save"].values():
            if save_method == "npy" and "numpy" not in dict_imports_merge:
                dict_imports_merge["numpy"] = "np"
            if save_method == "pkl" and "pickle" not in dict_imports_merge:
                dict_imports_merge["pickle"] = "pickle"

    # Get string imports
    str_imports = Block.get_external_l_imports_str(dict_imports_merge)

    # Build new blocks
    if "new_blocks" in master[gen]:
        for new_block_name, new_block in master[gen]["new_blocks"].items():

            # Get the list of final output # ! TODO convert to dict
            l_outputs_final = new_block["blocks"]['output']
            
            # Update parameters of each block to match the merged block specification
            l_blocks = []
            for block in new_block["blocks"]:
                block_to_update = copy.deepcopy(dict_blocks[block])
                l_params = new_block["blocks"][block]["params"]
                l_outputs = new_block["blocks"][block]["output"]
                block_to_update.parameters = OrderedDict(
                    [
                        (param, type_param)
                        for param, type_param in zip(l_params, block_to_update.parameters.values())
                    ]
                )
                block_to_update.dict_output = OrderedDict(
                    [
                        (output, type_output)
                        for output, type_output in zip(l_outputs, block_to_update.dict_output.values())
                    ]
                )
                l_blocks.append(block_to_update)
                
            

            new_block_function = Block.merge_blocks(
                l_blocks,
                new_block_name,
                docstring=new_block["docstring"],
                dict_output=new_block["blocks"]['output'],
            )

    # Get the dictionnary of block strings
    dict_blocks_str = {k: v.get_str() for k, v in dict_blocks.items()}

    # Get corresponding block string
    str_blocks = "\n".join([f"{k}" for k in dict_blocks_str.values()])

    # Handle output save
    if "save" in master[gen]:
        save = master[gen]["save"]
    else:
        save = None

    # Write main script
    str_main = generate_main(
        master[gen]["script"], config[gen], dict_blocks, save, dict_imports_merge
    )

    # Render template
    content = template.render(
        imports=str_imports,
        blocks=str_blocks,
        main=str_main,
    )
    with open(file_path, mode="w", encoding="utf-8") as file:
        file.write(content)

Now browsing generation: gen1




In [8]:
# Now with several blocks
from src import Block

dic_blocks["add"].parameters = OrderedDict([("a", dic_blocks["add"].output["c"]), ("output_factorial", dic_blocks["factorial"].output["output_factorial"])])
dic_blocks["multiply"].output = OrderedDict([("c", dic_blocks["multiply"].output["output_multiply"])])
dic_blocks["power"].output = OrderedDict([("d", dic_blocks["power"].output["output_power"])])
dic_blocks["print_result"]

test_multiple = Block.merge_blocks(
    [dic_blocks["factorial"], dic_blocks["add"], dic_blocks["multiply"], dic_blocks["power"], dic_blocks["print_result"]],
    "test_multiple_blocks",
    docstring="This function contains many blocks",
    output = dic_blocks["power"].output,
)

NameError: name 'dic_blocks' is not defined

In [12]:
print('*********')
print(test_multiple.get_str())
print('*********')
print(dic_blocks["add"].get_str())
print('*********')
print(test_multiple.get_output_str())
print('*********')
print(test_multiple.get_output_type_hint_str())
print('*********')
print(test_multiple.get_call_str())
print('*********')
print(test_multiple.get_assignation_call_str())
print('*********')
print(test_multiple.get_signature())
print('*********')
print(test_multiple.parameters)
print('*********')
print(test_multiple.dict_imports)
print('*********')
print(test_multiple.set_deps)

*********
def test_multiple_blocks(a: int, b: int) -> int:
    """This function contains many blocks
    """

    output_factorial = factorial_function(a)
    c = add_function(a, output_factorial)
    c = multiply_function(a, b)
    d = power_function(b, c)
    print_result_function(d)
    return d

*********
def add_function(a: int, output_factorial: int) -> int:
    """Dummy docstring
    """
    b = output_factorial
    
    # Add a and b
    return a + b

*********
d
*********
int
*********
test_multiple_blocks(a, b)
*********
d = test_multiple_blocks(a, b)
*********
(a: int, b: int) -> int
*********
OrderedDict([('a', <class 'int'>), ('b', <class 'int'>)])
*********
OrderedDict([('math', 'math'), ('numpy', 'np')])
*********
{'add_function', 'multiply_function', 'factorial_function', 'power_function', 'print_result_function'}
